## Creación de RDDs

### Usando SparkContext

El método sparkContext.parallelize nos permite crear un RDD a partir de una lista o una tupla

In [2]:
import pyspark
# Carga ufnciones extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession

In [3]:
sc = SparkSession.builder.appName('creando_rdds').getOrCreate()

24/02/24 11:56:32 WARN Utils: Your hostname, vania-Latitude-7400 resolves to a loopback address: 127.0.1.1; using 10.153.221.214 instead (on interface wlo1)
24/02/24 11:56:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/24 11:56:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/24 11:56:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/24 11:56:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
lista = ['en', 'un', 'lugar', 'de', 'un', 'gran', 'pais']

In [5]:
lista_rdd = sc.sparkContext.parallelize(lista, 4)

In [6]:
lista_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [7]:
lista_rdd.collect()

['en', 'un', 'lugar', 'de', 'un', 'gran', 'pais']

### Usando conjuntos de datos externos

A partir de una fuente de almacenamiento, se puede utilizar la función textFile del sparkContext

In [8]:
text = sc.sparkContext.textFile('../data/file.txt')

In [9]:
text

../data/file.txt MapPartitionsRDD[2] at textFile at NativeMethodAccessorImpl.java:0

In [10]:
text.collect()

['Hello World']

## Operaciones

### Acciones

In [11]:
rdd = sc.sparkContext.parallelize([4,1,2,6,1,5,3,3,2,4])

##### collect()

In [12]:
rdd.collect()

[4, 1, 2, 6, 1, 5, 3, 3, 2, 4]

In [13]:
lista = rdd.collect()
print('El tercer elemento de la lista es %d'% lista[2])

El tercer elemento de la lista es 2


In [14]:
lista

[4, 1, 2, 6, 1, 5, 3, 3, 2, 4]

##### count()

In [15]:
print('El RDD contiene %d elementos'% rdd.count())

El RDD contiene 10 elementos


##### countByValue()

In [16]:
rdd.countByValue()

defaultdict(int, {4: 2, 1: 2, 2: 2, 6: 1, 5: 1, 3: 2})

##### reduce()

Agrega los elementos de un RDD según la función que se le pase como parámetro. La función debe cumplir las siguientes propiedades para que pueda ser calculada en paralelo.

        A+B = B+A
        (A+B)+C = A+(B+C)
       
       
 Ejemplo: Multiplicar los valores de rdd y sumar los resultados

In [17]:
rdd2 = rdd.map(lambda x: x*2)
sum_total = rdd2.reduce(lambda x,y: x+y)

In [18]:
rdd.collect()

[4, 1, 2, 6, 1, 5, 3, 3, 2, 4]

In [19]:
rdd2.collect()

[8, 2, 4, 12, 2, 10, 6, 6, 4, 8]

In [20]:
sum_total

62

Ejemplo 2:  Crear un diccionario con elementos (x,1) y suma las apariciones por elemento

In [21]:
rdd_text = sc.sparkContext.parallelize(['red', 'red', 'blue', 'green', 'green', 'yellow'])
rdd_aux = rdd_text.map(lambda x: (x,1))
rdd_result = rdd_aux.reduceByKey(lambda x,y: x+y)

In [22]:
rdd_aux.collect()

[('red', 1),
 ('red', 1),
 ('blue', 1),
 ('green', 1),
 ('green', 1),
 ('yellow', 1)]

In [23]:
rdd_result.collect()

[('blue', 1), ('green', 2), ('red', 2), ('yellow', 1)]

#### foreach()

In [24]:
def impar(x):
    if x%2 == 1:
        print('%d es impar'% x)

rdd.foreach(impar)

3 es impar
5 es impar1 es impar

1 es impar
3 es impar


In [25]:
rdd.collect()

[4, 1, 2, 6, 1, 5, 3, 3, 2, 4]

##### collectAsMap()

In [26]:
sc.sparkContext.parallelize([('a','b'), ('c','d')]).collectAsMap()

{'a': 'b', 'c': 'd'}

In [27]:
sc.sparkContext.parallelize([('a','b','e'), ('c','d','f')]).collectAsMap()

ValueError: dictionary update sequence element #0 has length 3; 2 is required

### Transformaciones

##### map()

In [28]:
t1 = rdd.map(lambda x: x*2)
t1.collect()

[8, 2, 4, 12, 2, 10, 6, 6, 4, 8]

##### filter()

In [29]:
num = sc.sparkContext.parallelize([4,1,2,6,1,5,3, 1000, 100, 2000])
num.filter(lambda x: x < 50).collect()

[4, 1, 2, 6, 1, 5, 3]

#### distinct()

In [30]:
rdd.collect()

[4, 1, 2, 6, 1, 5, 3, 3, 2, 4]

In [31]:
rdd.distinct().collect()

[1, 2, 3, 4, 5, 6]

##### union()

In [32]:
city1 = sc.sparkContext.parallelize(['Barcelona', 'Madrid', 'Paris'])
city2 = sc.sparkContext.parallelize(['Madrid', 'Londres', 'Roma'])
city1.union(city2).collect()

['Barcelona', 'Madrid', 'Paris', 'Madrid', 'Londres', 'Roma']